In [1]:
%pip install -qq langchain langchain-nvidia-ai-endpoints gradio
%pip install -qq arxiv pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.7 MB/

In [2]:
from langchain_nvidia_ai_endpoints._common import NVEModel

In [6]:
import os
os.environ["NVIDIA_API_KEY"] = "nvapi-**********************************"


In [7]:

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [8]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

[Model(id='google/codegemma-7b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-codegemma-7b']),
 Model(id='google/deplot', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/google/deplot', aliases=['ai-google-deplot', 'playground_deplot', 'deplot']),
 Model(id='microsoft/phi-3-vision-128k-instruct', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/microsoft/phi-3-vision-128k-instruct', aliases=['ai-phi-3-vision-128k-instruct']),
 Model(id='meta/llama2-70b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-llama2-70b', 'playground_llama2_70b', 'llama2_70b', 'playground_llama2_13b', 'llama2_13b']),
 Model(id='adept/fuyu-8b', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/adept/fuyu-8b', aliases=['ai-fuyu-8b', 'playground_fuyu_8b', 'fuyu_8b']),
 Model(id='google/paligemma', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1

In [9]:
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

##**Chatting with Large Documents**

In [ ]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try:
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

NVIDIA API Key: ··········
Retrieved NVIDIA_API_KEY beginning with "nvapi-F82..."


{'ai-embed-qa-4': '09c64e32-2b65-4892-a285-2f585408d118',
 'ai-gemma-2b': '04174188-f742-4069-9e72-d77c2b77d3cb',
 'ai-sea-lion-7b-instruct': '02f84bf4-c1a1-489b-a9de-ac3e8dcdec14',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_yi_34b': '347fa3f3-d675-432c-b844-669ef8ee53df',
 'ai-rerank-qa-mistral-4b': '0bf77f50-5c35-4488-8e7a-f49bb1974af6',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'ai-arctic-embed-l': '1528a0ad-205a-46ac-a783-94e2372586a9',
 'ai-mixtral-8x22b': '39655fc1-9ebc-4b24-963e-6915ea6680de',
 'ai-phi-3-mini': '4a58c6cb-a9b4-4014-99de-3e704d4ae687',
 'playground_smaug_72b': '008cff6d-4f4c-4514-b61e-bcfad6ba52a7',
 'playground_gemma_7b': '1361fa56-61d7-4a12-af32-69a3825746fa',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'ai-microsoft-kosmos-2': '6018fed7-f227-48dc-99bc-3fd4264d5037',
 'ai-llama2-70b': '2fddadfb-7e76-4c8a-9b82-

### 1.Naive Approach: Stuff Your Document

Taking a representation of the document and feeding it all to a chat model! From a document perspective, this is known as document stuffing.



<!-- > <img src="https://drive.google.com/uc?export=view&id=14DRI_uDviqzqg14TKoIc8IlBc3Zsb8oO" width=800px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_stuff.png" width=800px/>
>

LangChain provides a simple framework for connecting LLMs to your own data sources via general chunking strategies and strong incorporation with embedding frameworks/services. This framework has initially grown around its strong general support for LLM features, which signals its active strengths closer to the chain abstractions and agent coordination.

LlamaIndex is a data framework for LLM applications to ingest, structure, and access private or domain-specific data. It has since branched out to include general LLM capabilities similar to LangChain, but as of now it is still strongest in addressing the document side of LLM components since its initial abstractions were centered around that problem.

###2. Loading Documents


LangChain provides a variety of document loaders to facilitate the injestion of various document formats (HTML, PDF, code) from many different sources and locations (local storage, private s3 buckets, public websites, messaging APIs, etc.). These loaders query your data sources and return a Document object which contains the content and metadata, usually in a plain-text or otherwise human-readible format. There are plenty of document loaders already built and ready to use, with the first-party LangChain options listed here.

In this example, we can load a research paper of our choice using one of the following LangChain loaders:


**UnstructuredFileLoader**: Generally-useful file loader for arbitrary files; doesn't make too many assumptions about your document structure and is usually sufficient.


**ArxivLoader**: A more specialized file-loader which can communicate with the Arxiv interface directly. Just one example of many, this will make some more assumptions about your data to yield nicer parsings and auto-fill metadata (useful when you have multiple documents/formats).
For our code example we will default to using ArxivLoader

In [11]:
%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs


In [12]:
%pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.4 MB/s eta 0:00:00


In [13]:
from langchain_community.document_loaders import ArxivLoader, UnstructuredFileLoader

In [ ]:
# documents = UnstructuredFileLoader("llama2_paper.pdf").load()

In [14]:
documents = ArxivLoader(query="1706.03762").load()  ## Attention is all you need

In [15]:
print("Number of Documents Retrieved:", len(documents))
print(f"Sample of Document 1 Content (Total Length: {len(documents[0].page_content)}):")
print(documents[0].page_content[:1000])

Number of Documents Retrieved: 1
Sample of Document 1 Content (Total Length: 39593):
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely

In [16]:
from pprint import pprint
pprint(documents[0].metadata)

{'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion '
            'Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin',
 'Published': '2023-08-02',
 'Summary': 'The dominant sequence transduction models are based on complex '
            'recurrent or\n'
            'convolutional neural networks in an encoder-decoder '
            'configuration. The best\n'
            'performing models also connect the encoder and decoder through an '
            'attention\n'
            'mechanism. We propose a new simple network architecture, the '
            'Transformer, based\n'
            'solely on attention mechanisms, dispensing with recurrence and '
            'convolutions\n'
            'entirely. Experiments on two machine translation tasks show these '
            'models to be\n'
            'superior in quality while being more parallelizable and requiring '
            'significantly\n'
            'less time to train. Our model achieves 28.4 BLEU


Though it may be tempting to accept the metadata format as-is and ignore the body entirely, there are a key selection of features that cannot be approached without diving into the full text:

The metadata is not guaranteed. In the case of arxiv, paper abstracts, titles, authors, and date are necessary components of a submission, so being able to query them is not surprising. For an arbitrary PDF or webpage though, the same is not necessarily the case.
The agent will not be able to go deeper into the document content. The summary is good to know and can be used as-is, but does not provide a straight-forward path to interacting with the body at any capacity (at least not from what we've learned).

The agent will still not be able to reason about too many documents at once. Perhaps two summaries into one context and ask some questions. But what happens when you need to interact with 5 documents at once? What about an entire directory? Very soon, you will notice that your context window will be overloaded with information just to summarize or even list out the documents you're interested in!

###3.Transforming The Documents

Once documents have been loaded, they often need to be transformed if we intend to pass them into our LLMs as context. One method of transformation is known as **chunking**, which breaks down large pieces of content into smaller segments. This technique is valuable because it helps [optimize the relevance of the content returned from the vector database](https://www.pinecone.io/learn/chunking-strategies/).

LangChain provides a [variety of document transformers](https://python.langchain.com/docs/integrations/document_transformers/) out of which we will use the [``RecursiveCharacterTextSplitter``](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter). This option will allow us tp split our document with preference for some natural stopping points that we want our chunks to follow (as much as possible).

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100, separators=["\n\n", "\n", ".", ";", ",", " ", ""],)

###Data Preprocessing

In [91]:
documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(documents)

In [92]:
def include_doc(doc):
    ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
    string = doc.page_content
    if len([l for l in string if l.isalpha()]) < (len(string)//2):
        return False
    return True

In [93]:
docs_split = [doc for doc in docs_split if include_doc(doc)]

In [88]:
print(len(docs_split))

37


In [89]:
for i in (0, 1, 2, 15, -1):
    print(f'[{i}] ', docs_split[i].page_content)
    print("-"*64)

[0]  Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. 

chunking is pretty naive, but highlights the ease of getting at least something working for our application.

<br>

Our approach for chunking is pretty naive, but highlights the ease of getting at least something working for our application. We made some effort to keep the chunk size small so that our models are able to wield it effectively as context, but how are we going to reason about all of these pieces?

**When extending and optimizing this approach for an arbitrary set of documents, some potential options include:**

- Identifying logical breaks or synthesis techniques (manually, automatically, LLM-assisted, etc).
- Aiming to construct chunks that are rich in unique and relevant information, avoiding redundancy to maximize database utility.
- Customizing chunking to fit the document’s nature, ensuring the chunks are contextually relevant and cohesive.
- Including key concepts, keywords, or metadata snippets in each chunk for improved searchability and relevance in the database.
- Continuously assessing chunking effectiveness and be ready to adjust strategies for optimal balance between size and content richness.
- Considering a hierarchy system (implicitly-generated or explicitly-specified) to improve retrieval attempts.
    - If interested, please look over the [**LlamaIndex tree structures from the index guide**](https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide.html#tree-index) as a starting point.

###4. Refining Summaries

In a persuit to reason about large amounts of documents automatically, one potential idea might be to use LLMs to create a dense summary or knowledge base. Similar to how we maintained a running history of the conversation via slot-filling in the previous notebook, is there any problem with keeping a running history of an entire document?



we'll be implementing a simple but useful Runnable that uses a while loop and the running state chain formulation to summarize a set of document chunks. This process is commonly known as "document refinement" and is largely akin to our previous conversation-focused slot-filling exercise; the only difference is that now we're dealing with a large document instead of a growing chat history.

<!-- > <img src="https://drive.google.com/uc?export=view&id=1J2XR8Cc8YSkVJMiJCknMkgA02mBT8riZ" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_refine.png" width=1000px/>

###The DocumentSummary BaseModel


Much like the `KnowledgeBase` class from the previous [notebook](https://github.com/kapardhi03/Rag-Agents/blob/main/Simple_RAG_Agents.ipynb), we can create a `DocumentSummaryBase` structure designed to encapsulate the essence of a document. The one below will use the `running_summary` field to query the model for a final summary while attempting to use the `main_ideas` and `loose_ends` fields as a bottleneck to keep the running summary from moving too fast. This is something we're going to have to enforce via prompt engineering, so the `summary_prompt` is also provided which shows how this information will be used. Feel free to modify it as necessary to make it work for your model of choice.

In [24]:
%pip install IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [41]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [27]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from IPython.display import clear_output

In [28]:
class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")

In [42]:
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are generating a running summary of the document. Make it readable by a technical user."
        " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
        " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
        " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    )), ('user', (
        "{format_instructions}. Follow the format precisely, including quotations and commas\n\n"
        "{info_base}\nWithout losing any of the info, update the knowledge base with the following: {input}"
    ))
])

In [43]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions': lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        return string
    return instruct_merge | prompt | llm | preparse | parser


In [69]:
latest_summary = ""
def RSummarizer(knowledge, llm, prompt, verbose=False):
    '''
    Exercise: Create a chain that summarizes
    '''
    def summarize_docs(docs):
        # Initialize the parse_chain appropriately; should include an RExtract instance.
        parse_chain = RunnableAssign({'info_base' : RExtract(knowledge.__class__, llm, prompt)})

        state = {'info_base' : knowledge}

        global latest_summary  ## If loop crashes, you can check out the latest_summary

        for i, doc in enumerate(docs):
            # Update the state as appropriate using your parse_chain component
            state['input'] = doc.page_content
            state = parse_chain.invoke(state)

            assert 'info_base' in state
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
    return RunnableLambda(summarize_docs)

In [97]:
instruct_model = ChatNVIDIA(model="llama2_70b") | StrOutputParser()
instruct_llm = instruct_model | PydanticOutputParser(pydantic_object=DocumentSummaryBase)

/usr/local/lib/python3.10/dist-packages/langchain_nvidia_ai_endpoints/_statics.py:313: UserWarning: Model llama2_70b is deprecated. Using meta/llama2-70b instead.
  warnings.warn(


In [103]:
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
try:
  summary = summarizer.invoke(docs_split[:15])
  # print(summary)
except Exception as e:
  print(e)

Invalid json output: running_summary: 'A new simple network architecture called the Transformer is proposed, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. It shows superior quality and parallelization in machine translation tasks.'

main_ideas:

1. The Transformer network architecture is proposed, based on attention mechanisms only.
2. It dispenses with recurrence and convolutions, making it more parallelizable.
3. It shows superior quality in machine translation tasks.

loose_ends:

1. The attention mechanism used in the Transformer architecture is not fully explained.
2. The paper does not provide a detailed comparison of the Transformer with other sequence transduction models.
3. The authors do not discuss the potential limitations of the Transformer architecture.


In [104]:
pprint(latest_summary)

DocumentSummaryBase(running_summary='', main_ideas=[], loose_ends=[])


###5. Synthetic Data Processing


#### **Generality of Refinement**

It's important to note that this "progressive summarization" technique is merely a starter chain that makes few assumptions about the initial data and desired output format. The same technique can be expanded far and wide to generate synthetic refinements with known metadata, active assumptions, and downstream objectives in mind.

**Consider these potential applications:**

1. **Aggregating Data**: Constructing structures that transform raw data from document chunks into coherent, useful summaries.
2. **Categorization and Sub-topic Analysis**: Creating systems that categorize insights from chunks into defined categories, tracking emerging sub-topics within each.
3. **Consolidation into Dense Informational Chunks**: Refining these structures to distill insights into compact segments, enriched with direct quotes for deeper analysis.

These applications hint at the creation of a **domain-specific knowledge graph** which can be accessed and traversed by a conversational chat model. Some utilities already exist to generate these automatically via tools like [**LangChain Knowledge Graphs**](https://python.langchain.com/docs/modules/memory/types/kg). Though you might need to develop hierarchical structures and tools to both construct and traverse such a structure, it is a viable option when you can properly refine a sufficient knowledge graph for your use case! For this intereted in more advanced knowledge graph construction techniques which rely on larger systems and vector similarity, we found the [**LangChain x Neo4j Article**](https://blog.langchain.dev/using-a-knowledge-graph-to-implement-a-devops-rag-application/) to be of interest.

